Install sqlite and import original datatset

In [ ]:
import sqlite3
import pandas as pd
import polars as pl

raw_df = pl.read_csv("new_iproperty_dataset.csv")

raw_pd_df = raw_df.to_pandas()

conn = sqlite3.connect("iproperty.db")

raw_pd_df.to_sql("raw_data", conn, if_exists="replace", index=False)


163537

Data cleaning Process

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import re

# ---------------------- Clean Data ----------------------

df = pd.read_csv('new_iproperty_dataset.csv')

# Rename Area to Area Code
df.rename(columns={'Area': 'Area Code'}, inplace=True)

# Clean Property Title
df['Property Title'] = df['Property Title'].str.split(',').str[0]
df['Property Title'] = df['Property Title'].str.title()

# Clean Property Price
df['Property Price'] = df['Property Price'].str.replace('RM', '', regex=False)
df['Property Price'] = df['Property Price'].str.replace(',', '', regex=False)
df['Property Price'] = pd.to_numeric(df['Property Price'], errors='coerce')
df.rename(columns={'Property Price': 'Property Price (RM)'}, inplace=True)

# Clean Property Location
split_cols = df['Property Location'].str.split(', ', expand=True)
split_cols.columns = ['Property Location (City)', 'Property Location (State)']
df.drop(columns=['Property Location'], inplace=True)
df.insert(2, 'Property Location (City)', split_cols['Property Location (City)'])
df.insert(3, 'Property Location (State)', split_cols['Property Location (State)'])

def parse_property_details(detail):
    if pd.isnull(detail):
        return pd.Series([None, None, None])

    # clean messy code（eg: Â â€¢Â ）
    clean_detail = re.sub(r'[^\x00-\x7F]+', ' ', str(detail))

    # extract Type（all character before Built-up）
    type_match = re.search(r"^(.*?)Built-up", clean_detail, re.IGNORECASE)
    if type_match:
        raw_type = type_match.group(1).strip()
        property_type = re.split(r'\s*\|\s*', raw_type)[0]
    else:
        property_type = None

    # extract Area
    area_match = re.search(r'Built[-\s]*up[^0-9]*([\d,]+)\s*sq\.?\s*ft', clean_detail, re.IGNORECASE)
    area = area_match.group(1).replace(',', '') if area_match else None

    # extract Furnishing status
    furnishing = "Unknown"
    if re.search(r'\bUnfurnished\b', clean_detail, re.IGNORECASE):
        furnishing = 'Unfurnished'
    elif re.search(r'\bPartially Furnished\b', clean_detail, re.IGNORECASE):
        furnishing = 'Partially Furnished'
    elif re.search(r'\bFully Furnished\b', clean_detail, re.IGNORECASE):
        furnishing = 'Fully Furnished'
    elif re.search(r'\bFurnished\b', clean_detail, re.IGNORECASE):
        furnishing = 'Furnished'

    return pd.Series([
        property_type.strip() if property_type else None,
        float(area) if area else None,
        furnishing
    ])

df[['Property Type', 'Property Size (sqft)', 'Property Furnishing Status']] = df['Property Details'].apply(parse_property_details)

# Delete Property Details
df = df.drop(columns=['Property Details'])

# Clean Property Agent
df['Property Agent'] = df['Property Agent'].str.title()
df['Property Agent'] = df['Property Agent'].apply(
    lambda x: None if re.search(r'\bsdn\.?\s*bhd\.?\b', str(x), re.IGNORECASE) else x
)

# Drop NaNs in price and Null in property type
df.dropna(subset=['Property Price (RM)'], inplace=True)
df.dropna(subset=['Property Type'], inplace=True)

# Drop illogical property size (lower than 70 sqft)
df = df[df['Property Size (sqft)'] >= 70]

# Drop duplicates
df.drop_duplicates(inplace=True)

# Print sample result
print(df.head())

      Area Code                                 Property Title  \
0  perlis-zop7y  Semi D 2 Tingkat - Taman Jaya Diri - Seriab -   
1  perlis-zop7y    Semi D 1 Tingkat - Taman Nyu Indah 2 - Arau   
2  perlis-zop7y                           Taman Seri Manis Dua   
3  perlis-zop7y   Teres 1 Tingkat - Bandar Baharu Putra Height   
4  perlis-zop7y   Teres 1 Tingkat - Bandar Baharu Putra Height   

  Property Location (City) Property Location (State)  Property Price (RM)  \
0                   Kangar                    Perlis             775776.0   
1                     Arau                    Perlis             398000.0   
2                   Kangar                    Perlis             306000.0   
3                     Arau                    Perlis             185000.0   
4                     Arau                    Perlis             210000.0   

  Property Agent                                         Source URL  \
0         Haneef  https://www.iproperty.com.my/sale/perlis-zop7y...  

Import cleaned dataset to sqlite db

In [ ]:
cleaned_pd_df = df.copy()

cleaned_pd_df.to_sql("cleaned_data", conn, if_exists="replace", index=False)


144214

Upload Database to google drive

In [ ]:
from google.colab import files
files.download("iproperty.db")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Apply SQL query

In [ ]:
#SELECT can be used wihtout cursor (optional)

sample = pd.read_sql_query("SELECT * FROM cleaned_data LIMIT 5;", conn)
print(sample)


           Area                                 Property Title  \
0  perlis-zop7y  Semi D 2 Tingkat - Taman Jaya Diri - Seriab -   
1  perlis-zop7y    Semi D 1 Tingkat - Taman Nyu Indah 2 - Arau   
2  perlis-zop7y                           Taman Seri Manis Dua   
3  perlis-zop7y   Teres 1 Tingkat - Bandar Baharu Putra Height   
4  perlis-zop7y   Teres 1 Tingkat - Bandar Baharu Putra Height   

  Property Location (City) Property Location (State)  Property Price (RM)  \
0                   Kangar                    Perlis             775776.0   
1                     Arau                    Perlis             398000.0   
2                   Kangar                    Perlis             306000.0   
3                     Arau                    Perlis             185000.0   
4                     Arau                    Perlis             210000.0   

  Property Agent                                         Source URL  \
0         Haneef  https://www.iproperty.com.my/sale/perlis-zop7y...  

In [ ]:
#Create a cursor for executing sql query

cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM cleaned_data")
print("Rows in raw_listings:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM raw_data")
print("Rows in raw_listings:", cursor.fetchone()[0])

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print("Tables:", cursor.fetchall())

Rows in raw_listings: 150195
Rows in raw_listings: 163537
Tables: [('raw_data',), ('cleaned_data',)]


Delete empty value from property type

In [ ]:
query = '''
DELETE FROM cleaned_data
WHERE "Property Type" IS NULL;
'''

cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

Average property price in each state


In [ ]:
query = '''
SELECT "Property Location (State)" AS State,
ROUND(AVG("Property Price (RM)"), 2) AS AveragePrice
FROM cleaned_data
GROUP BY "Property Location (State)"
ORDER BY AveragePrice DESC;
'''

"""
Using pandas directly

result = pd.read_sql_query(query, conn)
print(result)
"""

cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

conn.close()

('Perak', 7335656.79)
('Putrajaya', 3026249.66)
('Kuala Lumpur', 2154244.93)
('Penang', 2086969.23)
('Selangor', 1543564.15)
('Sabah', 1141184.59)
('Johor', 997651.26)
('Sarawak', 982912.26)
('Pahang', 798681.59)
('Negeri Sembilan', 710603.88)
('Terengganu', 666883.17)
('Kedah', 639914.98)
('Melaka', 622933.8)
('Kelantan', 571128.42)
('Perlis', 375899.84)


Top 10 city with most property

In [ ]:
#Reopen the database if closed it before

conn = sqlite3.connect("iproperty.db")
cursor = conn.cursor()

query = '''
SELECT "Property Location (City)", COUNT(*) AS PropertyCount
FROM cleaned_data
GROUP BY "Property Location (City)"
ORDER BY PropertyCount DESC
LIMIT 10;
'''

cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

conn.close()

('Cheras', 3855)
('Ampang', 3747)
('Kepong', 2271)
('Ulu Kelang', 2151)
('Johor Bahru', 1957)
('Skudai', 1954)
('Iskandar Puteri (Nusajaya)', 1950)
('Tebrau', 1942)
('Ipoh', 1936)
('Shah Alam', 1935)
